In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir('/content/drive/MyDrive/hw6')

Mounted at /content/drive/


WE WANT TO PREDICT ARGUMENTS 'ARG1' FROM SENTENCES FROM NOMBANK DATASET

In [ ]:
class Token():
    def __init__(self, word, pos, bio,wordid,sentid,role):
        self.token = word
        self.POS = pos
        self.BIO = bio
        self.wordid=wordid
        self.sentid=sentid
        self.role=role

        self.previous_word = ''
        self.previous_pos = ''

        self.following_word = ''
        self.following_pos = ''

        self.previous_2_word = ''
        self.previous_2_pos = ''

        self.following_2_word = ''
        self.following_2_pos = ''

In [ ]:
# Features like 'word','pos','previous_word','previous_pos','previous_2_word','previous_2_pos','following_word','following_pos','following_2_word','following_2_pos',
#'isPred','wordId','sentId','bio'
# this class preprocesses the features

class Featuring():
    def __init__(self, token_list):
        self.token_list = token_list

    def update_attributes(self):
        for i in range(0, len(self.token_list)):
            token = self.token_list[i]
            if token.token == '':
                continue
            else:
                if self.token_list[i-1].token == '':
                    token.previous_word = 'Begin_of_Sent'
                    token.previous_pos = 'Begin_of_Sent'
                    token.previous_2_pos = 'Begin_of_Sent'
                    token.previous_2_word = 'Begin_of_Sent'
                else:
                    token.previous_word = self.token_list[i-1].token
                    token.previous_pos = self.token_list[i-1].POS
                    if self.token_list[i-2].token == '':
                        token.previous_2_pos = 'Begin_of_Sent'
                        token.previous_2_word = 'Begin_of_Sent'
                    else:
                        token.previous_2_pos = self.token_list[i-2].POS
                        token.previous_2_word = self.token_list[i-2].token

                if self.token_list[i+1].token == '':
                    token.following_word = 'End_of_Sent'
                    token.following_pos = 'End_of_Sent'
                    token.following_2_word = 'End_of_Sent'
                    token.following_2_pos = 'End_of_Sent'
                else:
                    token.following_word = self.token_list[i+1].token
                    token.following_pos = self.token_list[i+1].POS
                    if self.token_list[i+2].token == '':
                        token.following_2_pos = 'Begin_of_Sent'
                        token.following_2_word = 'Begin_of_Sent'
                    else:
                        token.following_2_pos = self.token_list[i+2].POS
                        token.following_2_word = self.token_list[i+2].token

        return
    
    def output(self, filename,pred_file_name,machine_file_name):
        file = open(filename, 'w')
        pred_file = open(pred_file_name, 'w')
        machine_file=open(machine_file_name, 'w')
       

        pred_file.write('Label'+'\t'+'sentId'+'\n')
        machine_file.write('word' + '\t'+'pos'+ '\t'+'previous_word'+ '\t'+'previous_pos'+'\t'+'previous_2_word'+ 
                           '\t'+'previous_2_pos' + '\t'+'following_word'+ '\t'+'following_pos' + '\t'+'following_2_word'+ '\t' + 'following_2_pos'+'\t'+'isPred'+'\t'+'wordId'+'\t'+'sentId'+'\t'+'bio'+'\n')
        for token in self.token_list:
            
            role='0'
            if token.role=='PRED':
               role='1'

            if token.token == '':
                file.write('\n')
                pred_file.write('\n')
                machine_file.write('\n')
            else:
                str1 = token.token + '\t'+'pos='+token.POS + '\t'+\
                      'previous_word='+token.previous_word + '\t'+'previous_pos='+token.previous_pos + '\t'+\
                      'previous_2_word='+token.previous_2_word + '\t'+'previous_2_pos='+token.previous_2_pos + '\t'+\
                      'following_word='+token.following_word + '\t'+'following_pos='+token.following_pos + '\t'+\
                      'following_2_word='+ token.following_2_word + '\t' + 'following_2_pos=' + token.following_2_pos+ '\t' + 'isPred=' + role

                
                machine_str = token.token + '\t'+token.POS + '\t'+token.previous_word + '\t'+token.previous_pos + '\t'+token.previous_2_word + '\t'+token.previous_2_pos + '\t'+token.following_word + '\t'+token.following_pos + '\t'+ token.following_2_word +'\t'+ token.following_2_pos +'\t'+ role+'\t'+ str(token.wordid)+'\t'+str(token.sentid)
                
                if token.BIO == '':
                    str1 += '\n'
                    machine_str+= '\n'
                    file.write(str1)
                    machine_file.write(machine_str)
                else:
                    str1 += '\t' + token.BIO + '\n'
                    machine_str += '\t' + token.BIO + '\n'
                    file.write(str1)
                    machine_file.write(machine_str)
                
                if token.role=='ARG1':
                  strr='1'+'\t'+str(token.sentid)+'\n'
                  print(strr)
                  pred_file.write(strr)
                else:
                  strr='0'+'\t'+str(token.sentid)+'\n'
                  pred_file.write(strr)
            
        file.close()

In [ ]:
def main():
    file = open('%_nombank.clean.train', 'r')
    input_list = file.read().split('\n')
    file.close()
    print(input_list[0])
    token_list = []
    for token in input_list:
        if token == '':
            token_list.append(Token('', '', '','','',''))
        else:
            temp = token.split()
            if len(temp)>5:
              token_list.append(Token(temp[0], temp[1], temp[2],temp[3],temp[4],temp[5]))
            else:
              token_list.append(Token(temp[0], temp[1], temp[2],temp[3],temp[4],''))
    print('token_list generated...')
    process = Featuring(token_list)
    print('featuring set up...')
    process.update_attributes()
    print('attributes updated...')
    process.output('training.feature','pred_file','df_file')
    print('training feature done...')

    file = open('%_nombank.clean.test', 'r')
    input_list = file.read().split('\n')
    file.close()
    print(input_list[0])
    token_list = []
    for token in input_list:
        if token == '':
            token_list.append(Token('', '', '','','',''))
        else:
            temp = token.split()
            if len(temp)>5:
              token_list.append(Token(temp[0], temp[1], temp[2],temp[3],temp[4],temp[5]))
            else:
              token_list.append(Token(temp[0], temp[1], temp[2],temp[3],temp[4],''))
    print('token_list generated...')
    process = Featuring(token_list)
    print('featuring set up...')
    process.update_attributes()
    print('attributes updated...')
    process.output('testing.feature','test_pred_file','test_file')
    print('training feature done...')

In [ ]:
main()

In [ ]:
import pandas as pd
df=pd.read_csv('df_file', sep='\t', lineterminator='\n',index_col=False)
pred_df=pd.read_csv('pred_file', sep='\t', lineterminator='\n',index_col=False)
test_pred_df=pd.read_csv('test_pred_file', sep='\t', lineterminator='\n',index_col=False)
test_file=pd.read_csv('test_file', sep='\t', lineterminator='\n',index_col=False)

In [ ]:
pred_df.head()

,Label,sentId
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [ ]:
df.fillna('False')
test_file.fillna('False')

,word,pos,previous_word,previous_pos,previous_2_word,previous_2_pos,following_word,following_pos,following_2_word,following_2_pos,isPred,wordId,sentId,bio
0,Then,RB,Begin_of_Sent,Begin_of_Sent,Begin_of_Sent,Begin_of_Sent,in,IN,a,DT,0,0,0,B-ADVP
1,in,IN,Then,RB,Begin_of_Sent,Begin_of_Sent,a,DT,lightning,NN,0,1,0,B-PP
2,a,DT,in,IN,Then,RB,lightning,NN,plunge,NN,0,2,0,B-NP
3,lightning,NN,a,DT,in,IN,plunge,NN,COMMA,COMMA,0,3,0,I-NP
4,plunge,NN,lightning,NN,a,DT,COMMA,COMMA,the,DT,0,4,0,I-NP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4273,of,IN,%,NN,160,CD,the,DT,PSA,NNP,0,29,149,B-PP
4274,the,DT,of,IN,%,NN,PSA,NNP,model,NN,0,30,149,B-NP
4275,PSA,NNP,the,DT,of,IN,model,NN,.,.,0,31,149,I-NP
4276,model,NN,PSA,NNP,the,DT,.,.,Begin_of_Sent,Begin_of_Sent,0,32,149,I-NP


In [ ]:
df.head()

,word,pos,previous_word,previous_pos,previous_2_word,previous_2_pos,following_word,following_pos,following_2_word,following_2_pos,isPred,wordId,sentId,bio
0,But,CC,Begin_of_Sent,Begin_of_Sent,Begin_of_Sent,Begin_of_Sent,about,IN,25,CD,0,0,0,O
1,about,IN,But,CC,Begin_of_Sent,Begin_of_Sent,25,CD,%,NN,0,1,0,B-NP
2,25,CD,about,IN,But,CC,%,NN,of,IN,0,2,0,I-NP
3,%,NN,25,CD,about,IN,of,IN,the,DT,1,3,0,I-NP
4,of,IN,%,NN,25,CD,the,DT,insiders,NNS,0,4,0,B-PP


In [ ]:
combined_df=df.append(test_file)
df=combined_df
print(df)

       word  pos  previous_word   previous_pos previous_2_word previous_2_pos  \
0       But   CC  Begin_of_Sent  Begin_of_Sent   Begin_of_Sent  Begin_of_Sent   
1     about   IN            But             CC   Begin_of_Sent  Begin_of_Sent   
2        25   CD          about             IN             But             CC   
3         %   NN             25             CD           about             IN   
4        of   IN              %             NN              25             CD   
...     ...  ...            ...            ...             ...            ...   
4273     of   IN              %             NN             160             CD   
4274    the   DT             of             IN               %             NN   
4275    PSA  NNP            the             DT              of             IN   
4276  model   NN            PSA            NNP             the             DT   
4277      .    .          model             NN             PSA            NNP   

     following_word followi

In [ ]:
end_no=len(test_file)
print(end_no)

4278


In [ ]:
df.columns

Index(['word', 'pos', 'previous_word', 'previous_pos', 'previous_2_word',
       'previous_2_pos', 'following_word', 'following_pos', 'following_2_word',
       'following_2_pos', 'isPred', 'wordId', 'sentId', 'bio'],
      dtype='object')

In [ ]:
pred_df

,Label,sentId
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
61126,0,2173
61127,0,2173
61128,0,2173
61129,0,2173


In [ ]:
from sklearn import preprocessing

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
df['word']= label_encoder.fit_transform(df['word'])
df['pos']= label_encoder.fit_transform(df['pos'])
df['previous_word']= label_encoder.fit_transform(df['previous_word'])
df['previous_pos']= label_encoder.fit_transform(df['previous_pos'])
df['previous_2_word']= label_encoder.fit_transform(df['previous_2_word'])
df['previous_2_pos']= label_encoder.fit_transform(df['previous_2_pos'])
df['following_word']= label_encoder.fit_transform(df['following_word'])
df['following_pos']= label_encoder.fit_transform(df['following_pos'])
df['following_2_word']= label_encoder.fit_transform(df['following_2_word'])
df['following_2_pos']= label_encoder.fit_transform(df['following_2_pos'])
df['bio']= label_encoder.fit_transform(df['bio'])
df['isPred']= label_encoder.fit_transform(df['isPred'])
df['wordId']= label_encoder.fit_transform(df['wordId'])
df['sentId']= label_encoder.fit_transform(df['sentId'])

pred_df['Label']=label_encoder.fit_transform(pred_df['Label'])

print(df.shape)

(65409, 14)


In [ ]:
test_file=df.iloc[61131:]
df=df.iloc[0:61131]
print(len(test_file))
print(len(df))

4278
61131


In [ ]:
print(df.head())
print(pred_df)

   word  pos  previous_word  previous_pos  previous_2_word  previous_2_pos  \
0  1790    9           1711             9             1663               9   
1  3547   15           1791            10             1663               9   
2   639   10           3548            16             1742              10   
3     2   20            639            11             3455              16   
4  5774   15              2            21              626              11   

   following_word  following_pos  following_2_word  following_2_pos  isPred  \
0            3153             16               632               11       0   
1             639             10                 2               22       0   
2               2             21              5214               17       0   
3            5380             16              6351               13       1   
4            6534             12              4693               25       0   

   wordId  sentId  bio  
0       0       0   12  
1     

In [ ]:

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

test_pred_df['Label']=label_encoder.fit_transform(test_pred_df['Label'])
test_pred_df['sentId']=label_encoder.fit_transform(test_pred_df['sentId'])
pred_df['sentId']=label_encoder.fit_transform(pred_df['sentId'])
print(test_file.shape)

(4278, 14)


In [ ]:
df.head()

,word,pos,previous_word,previous_pos,previous_2_word,previous_2_pos,following_word,following_pos,following_2_word,following_2_pos,isPred,wordId,sentId,bio
0,1790,9,1711,9,1663,9,3153,16,632,11,0,0,0,12
1,3547,15,1791,10,1663,9,639,10,2,22,0,1,0,3
2,639,10,3548,16,1742,10,2,21,5214,17,0,2,0,10
3,2,20,639,11,3455,16,5380,16,6351,13,1,3,0,10
4,5774,15,2,21,626,11,6534,12,4693,25,0,4,0,4


In [ ]:
pred_df.shape

(61131, 2)

In [ ]:
X_Train1 = df.groupby(
['sentId'],as_index=False)['word','pos','previous_word','previous_pos','previous_2_word','previous_2_pos','following_word','following_pos','following_2_word','following_2_pos','isPred','wordId','bio'].agg(lambda x: list(x)).drop(columns='sentId')

<ipython-input-14-ca3c6ba983a5>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  X_Train1 = df.groupby(


In [ ]:
ll=X_Train1.iloc[0]['word']

In [ ]:
print(X_Train1.head())

                                                word  \
0  [1790, 3547, 639, 2, 5774, 6928, 5247, 1804, 3...   
1  [3346, 3930, 4976, 7241, 5290, 5196, 5110, 582...   
2  [3346, 6022, 5774, 6928, 3437, 3264, 1804, 722...   
3  [3346, 4883, 5173, 4311, 6512, 6118, 7137, 694...   
4  [2763, 2586, 6464, 6928, 5173, 6222, 4349, 724...   

                                                 pos  \
0  [9, 15, 10, 20, 15, 12, 23, 11, 36, 15, 21, 23...   
1  [12, 20, 20, 19, 34, 15, 16, 9, 16, 23, 11, 23...   
2  [12, 20, 15, 12, 21, 22, 11, 40, 28, 35, 10, 2...   
3  [12, 16, 16, 20, 23, 37, 36, 12, 16, 20, 35, 3...   
4  [21, 21, 35, 12, 20, 23, 37, 15, 12, 16, 20, 3...   

                                       previous_word  \
0  [1711, 1791, 3548, 639, 2, 5775, 6929, 5248, 1...   
1  [1711, 3347, 3931, 4977, 7242, 5291, 5197, 511...   
2  [1711, 3347, 6023, 5775, 6929, 3438, 3265, 180...   
3  [1711, 3347, 4884, 5174, 4312, 6513, 6119, 713...   
4  [1711, 2764, 2587, 6465, 6929, 5174, 6223, 

In [ ]:
Y_Train1 = pred_df.groupby(['sentId'],as_index=False)['Label'].agg(lambda x: list(x)).drop(columns='sentId')

In [ ]:
len(Y_Train1)

2174

In [ ]:
len(Y_Train1.iloc[0]['Label'])

18

In [ ]:
from keras.utils import to_categorical

In [ ]:
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
#n_token = len(list(set(df['word'].to_list())))
n_token = len(list(set(combined_df['word'].to_list())))
n_tag = len(list(set(pred_df['Label'].to_list())))

tokens = X_Train1['word'].tolist()
maxlen = max([len(s) for s in tokens])
pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

tags = Y_Train1['Label'].tolist()
pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= 0)
pad_tags = [to_categorical(i, num_classes=2) for i in pad_tags]

train_tokens, val_tokens, train_tags, val_tags = train_test_split(pad_tokens,pad_tags,test_size = 0.25,train_size =0.75, random_state=2020)

In [ ]:
n_pos = len(list(set(combined_df['pos'].to_list())))

poss = X_Train1['pos'].tolist()
maxlenpos = max([len(s) for s in poss])
pad_pos = pad_sequences(poss, maxlen=maxlenpos, dtype='int32', padding='post', value= n_pos - 1)

train_pos, val_pos = train_test_split(pad_pos,test_size = 0.25,train_size =0.75, random_state=2020)

In [ ]:
n_bio = len(list(set(combined_df['bio'].to_list())))

bios = X_Train1['bio'].tolist()
maxlenbio = max([len(s) for s in bios])
pad_bio = pad_sequences(bios, maxlen=maxlenbio, dtype='int32', padding='post', value= n_bio - 1)

train_bio, val_bio = train_test_split(pad_bio,test_size = 0.25,train_size =0.75, random_state=2020)

In [ ]:
print(n_bio, n_pos)

13 45


In [ ]:
print(train_bio.shape)
print(train_pos.shape)
print(len(train_tags))
print(train_tokens.shape)

(1630, 67)
(1630, 67)
1630
(1630, 67)


In [ ]:
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate

token_input = Input(shape=(None,), dtype='int32', name='token_input')
token_embedding = Embedding(input_dim=50000,output_dim=train_tokens.shape[1])(token_input)

pos_input = Input(shape=(None,),dtype='int32',name='pos_input')
pos_embedding = Embedding(input_dim=50000, output_dim=train_pos.shape[1])(pos_input)


bio_input = Input(shape=(None,), dtype='int32', name='bio_input')
bio_embedding = Embedding(input_dim=50000,output_dim=train_bio.shape[1])(bio_input)


In [ ]:
concatenate_output = concatenate([token_embedding, pos_embedding,bio_embedding])

In [ ]:
concatenate_output.shape

TensorShape([None, None, 201])

In [ ]:
n_tags=2

In [ ]:
from keras.models import Model
output = Bidirectional(LSTM(67, return_sequences=True, dropout=0.2, recurrent_dropout=0.25), merge_mode = 'concat')(concatenate_output)
output = TimeDistributed(Dense(n_tags, activation="softmax"))(output)
model = Model(inputs=[token_input,pos_input,bio_input], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 token_input (InputLayer)       [(None, None)]       0           []                               
                                                                                                  
 pos_input (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 bio_input (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 67)     3350000     ['token_input[0][0]']            
                                                                                              

In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras.utils import plot_model
def train_model(X, y,X_val,y_val, model):
    loss = list()
    valloss = list()
    acc = list()
    valacc = list()
    for i in range(150):
        print("current epoch::",i)
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, validation_data=(X_val,y_val),batch_size=1000, verbose=1, epochs=1)
        
        loss.append(hist.history['loss'][0])
        valloss.append(hist.history['val_loss'][0])
        acc.append(hist.history['accuracy'][0])
        valacc.append(hist.history['val_accuracy'][0])
    return loss,valloss,acc,valacc

plot_model(model)
loss,valloss,acc,valacc = train_model([train_tokens,train_pos,train_bio], np.array(train_tags), [val_tokens,val_pos,val_bio], np.array(val_tags), model=model)

current epoch:: 0
2/2 [==============================] - 8s 3s/step - loss: 0.0086 - accuracy: 0.9968 - val_loss: 0.0340 - val_accuracy: 0.9899
current epoch:: 1
2/2 [==============================] - 8s 3s/step - loss: 0.0084 - accuracy: 0.9969 - val_loss: 0.0343 - val_accuracy: 0.9899
current epoch:: 2
2/2 [==============================] - 8s 3s/step - loss: 0.0084 - accuracy: 0.9970 - val_loss: 0.0326 - val_accuracy: 0.9898
current epoch:: 3
2/2 [==============================] - 8s 3s/step - loss: 0.0089 - accuracy: 0.9968 - val_loss: 0.0333 - val_accuracy: 0.9898
current epoch:: 4
2/2 [==============================] - 8s 3s/step - loss: 0.0088 - accuracy: 0.9967 - val_loss: 0.0334 - val_accuracy: 0.9898
current epoch:: 5
2/2 [==============================] - 8s 3s/step - loss: 0.0086 - accuracy: 0.9969 - val_loss: 0.0329 - val_accuracy: 0.9898
current epoch:: 6
2/2 [==============================] - 8s 3s/step - loss: 0.0083 - accuracy: 0.9968 - val_loss: 0.0343 - val_accuracy:

In [ ]:
model.save("models/softmax_no_lstm_150.h6")

In [ ]:
from tensorflow import keras
model = keras.models.load_model('models/hw6_1.h6')

In [ ]:
X_Test = test_file.groupby(
['sentId'],as_index=False)['word','pos','previous_word','previous_pos','previous_2_word','previous_2_pos','following_word','following_pos','following_2_word','following_2_pos','isPred','wordId','bio'].agg(lambda x: list(x)).drop(columns='sentId')

Y_Test = test_pred_df.groupby(['sentId'],as_index=False)['Label'].agg(lambda x: list(x)).drop(columns='sentId')

tokens = X_Test['word'].tolist()
maxlen = max([len(s) for s in tokens])
test_tokens = pad_sequences(tokens, maxlen=67, dtype='int32', padding='post', value= n_token - 1)

tags = Y_Test['Label'].tolist()
pad_tags = pad_sequences(tags, maxlen=67, dtype='int32', padding='post', value= 0)
test_tags = [to_categorical(i, num_classes=2) for i in pad_tags]

poss = X_Test['pos'].tolist()
maxlenpos = max([len(s) for s in poss])
test_pos = pad_sequences(poss, maxlen=67, dtype='int32', padding='post', value= n_pos - 1)

bios = X_Test['bio'].tolist()
maxlenbio = max([len(s) for s in bios])
test_bio = pad_sequences(bios, maxlen=67, dtype='int32', padding='post', value= n_bio - 1)


<ipython-input-46-a1d69a2e5b27>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  X_Test = test_file.groupby(


In [ ]:
print(test_bio.shape)
print(test_pos.shape)
print(len(test_tags))
print(test_tokens.shape)

(150, 67)
(150, 67)
150
(150, 67)


In [ ]:
Y_pred=model.predict([test_tokens,test_pos,test_bio])

5/5 [==============================] - 0s 55ms/step


In [ ]:
print(Y_pred[1])

In [ ]:
dictt={}
cnt=-1
for i in test_tags:
  cnt=cnt+1
  ccnt=-1
  for temp in i:
    ccnt=ccnt+1
    if temp[1] == 1.0:
      dictt[cnt]=ccnt

print(dictt)

{0: 9, 1: 19, 2: 1, 3: 16, 4: 2, 5: 3, 6: 2, 7: 10, 8: 16, 9: 15, 10: 15, 11: 16, 12: 17, 13: 10, 14: 21, 15: 14, 16: 14, 17: 29, 18: 12, 19: 23, 20: 13, 21: 21, 22: 17, 23: 12, 24: 14, 25: 25, 26: 15, 27: 4, 28: 4, 29: 26, 30: 28, 31: 31, 32: 2, 33: 8, 34: 8, 35: 4, 36: 0, 37: 29, 38: 31, 39: 43, 40: 25, 41: 34, 42: 42, 43: 18, 44: 12, 45: 12, 46: 14, 47: 1, 48: 1, 49: 16, 50: 22, 51: 22, 52: 5, 53: 29, 54: 29, 55: 7, 56: 0, 57: 8, 58: 1, 59: 5, 60: 12, 61: 17, 62: 24, 63: 13, 64: 8, 65: 8, 66: 7, 67: 26, 68: 11, 69: 15, 70: 6, 71: 6, 72: 2, 73: 2, 74: 6, 75: 11, 76: 6, 77: 6, 78: 4, 79: 6, 80: 43, 81: 10, 82: 16, 83: 16, 84: 2, 85: 2, 86: 13, 87: 0, 88: 1, 89: 1, 90: 9, 91: 0, 92: 16, 93: 16, 94: 0, 95: 1, 96: 0, 97: 0, 98: 0, 99: 26, 100: 26, 101: 5, 102: 20, 103: 41, 104: 31, 105: 23, 106: 3, 107: 1, 108: 21, 109: 7, 110: 9, 111: 7, 112: 21, 113: 14, 114: 6, 115: 4, 116: 8, 117: 11, 118: 14, 119: 1, 120: 7, 121: 4, 122: 35, 123: 11, 124: 2, 125: 5, 126: 3, 127: 3, 128: 0, 129: 1, 1

In [ ]:
sum=0
for key in dictt.keys():
  if Y_pred[key][dictt.get(key)][1]>0.5: # printing only the value of place value with  > 0.5
    print("sentence no::",key)
    print("predicted_values::",Y_pred[key][dictt.get(key)])
    sum=sum+1
print(sum)

sentence no:: 0
predicted_values:: [0.05582246 0.94417745]
sentence no:: 4
predicted_values:: [0.0235378 0.9764622]
sentence no:: 5
predicted_values:: [0.1403861 0.8596139]
sentence no:: 6
predicted_values:: [0.20596187 0.79403806]
sentence no:: 7
predicted_values:: [0.4226221  0.57737786]
sentence no:: 11
predicted_values:: [0.00215753 0.9978425 ]
sentence no:: 13
predicted_values:: [0.03802133 0.9619787 ]
sentence no:: 15
predicted_values:: [0.07399856 0.9260014 ]
sentence no:: 16
predicted_values:: [0.07399856 0.9260014 ]
sentence no:: 17
predicted_values:: [0.09942973 0.90057033]
sentence no:: 18
predicted_values:: [0.08968418 0.9103159 ]
sentence no:: 19
predicted_values:: [0.13639887 0.86360115]
sentence no:: 20
predicted_values:: [0.06841893 0.9315811 ]
sentence no:: 21
predicted_values:: [0.37759614 0.62240386]
sentence no:: 23
predicted_values:: [0.0971339  0.90286607]
sentence no:: 25
predicted_values:: [0.2785944  0.72140557]
sentence no:: 27
predicted_values:: [0.00143176 0

In [ ]:
len(Y_list)

10050

In [ ]:
Y_list=[]

for ll in Y_pred:
  for ll2 in ll:
    #print(ll2)
    if ll2[0]>0.5:
      Y_list.append(0)
    else:
      Y_list.append(1)

In [ ]:
Y_test_list=[]

for ll in test_tags:
  for ll2 in ll:
    #print(ll2)
    if ll2[0]==1.0:
      Y_test_list.append(0)
    else:
      Y_test_list.append(1)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['Not Arg1', 'Arg1']
print(classification_report(Y_test_list, Y_list, target_names=target_names))

              precision    recall  f1-score   support

    Not Arg1       0.99      1.00      1.00      9899
        Arg1       0.74      0.61      0.67       151

    accuracy                           0.99     10050
   macro avg       0.87      0.80      0.83     10050
weighted avg       0.99      0.99      0.99     10050



In [ ]:
y_pred_dict={}
cnt=-1
for i in Y_pred:
  cnt=cnt+1
  ccnt=-1
  for temp in i:
    ccnt=ccnt+1
    if temp[1] > 0.5:
      if cnt in y_pred_dict.keys():
        y_pred_dict[cnt].append(ccnt)
      else:
        y_pred_dict[cnt]=[ccnt]

In [ ]:
print(y_pred_dict)

{0: [9, 40], 3: [16, 21], 4: [2], 5: [3], 6: [2], 7: [10], 8: [26], 11: [16], 13: [10], 14: [14, 21], 15: [14, 21], 16: [14, 21], 17: [29], 18: [12], 19: [23], 20: [13], 21: [21], 23: [12], 25: [25], 27: [4], 28: [4], 29: [26], 30: [28], 32: [6], 33: [8], 34: [8], 35: [16], 36: [0], 38: [31], 39: [43], 43: [18], 44: [12], 45: [12], 50: [13, 22], 51: [13, 22], 53: [29], 54: [29], 55: [7], 56: [0], 57: [7, 8], 58: [1], 59: [2, 5], 60: [12], 61: [14, 17], 62: [13], 63: [13], 64: [8], 65: [8], 66: [7], 67: [20, 26], 68: [11], 69: [15], 70: [6], 71: [6], 72: [2], 73: [2], 75: [11], 79: [6], 80: [43], 81: [10], 84: [2], 85: [2], 86: [2], 87: [2], 88: [1], 89: [1], 90: [4, 9], 91: [0], 92: [0], 93: [0], 94: [0], 95: [1], 99: [26], 100: [26], 101: [5, 12], 103: [41], 104: [31], 105: [23], 106: [3], 107: [1], 108: [21], 109: [21], 110: [9, 25], 111: [3, 7], 112: [21], 116: [8, 16], 117: [11], 118: [6], 119: [1, 3], 120: [7], 122: [35], 124: [2], 125: [5], 126: [3], 129: [1], 130: [8, 18], 131: 

In [ ]:
print(dictt)

{0: 9, 1: 19, 2: 1, 3: 16, 4: 2, 5: 3, 6: 2, 7: 10, 8: 16, 9: 15, 10: 15, 11: 16, 12: 17, 13: 10, 14: 21, 15: 14, 16: 14, 17: 29, 18: 12, 19: 23, 20: 13, 21: 21, 22: 17, 23: 12, 24: 14, 25: 25, 26: 15, 27: 4, 28: 4, 29: 26, 30: 28, 31: 31, 32: 2, 33: 8, 34: 8, 35: 4, 36: 0, 37: 29, 38: 31, 39: 43, 40: 25, 41: 34, 42: 42, 43: 18, 44: 12, 45: 12, 46: 14, 47: 1, 48: 1, 49: 16, 50: 22, 51: 22, 52: 5, 53: 29, 54: 29, 55: 7, 56: 0, 57: 8, 58: 1, 59: 5, 60: 12, 61: 17, 62: 24, 63: 13, 64: 8, 65: 8, 66: 7, 67: 26, 68: 11, 69: 15, 70: 6, 71: 6, 72: 2, 73: 2, 74: 6, 75: 11, 76: 6, 77: 6, 78: 4, 79: 6, 80: 43, 81: 10, 82: 16, 83: 16, 84: 2, 85: 2, 86: 13, 87: 0, 88: 1, 89: 1, 90: 9, 91: 0, 92: 16, 93: 16, 94: 0, 95: 1, 96: 0, 97: 0, 98: 0, 99: 26, 100: 26, 101: 5, 102: 20, 103: 41, 104: 31, 105: 23, 106: 3, 107: 1, 108: 21, 109: 7, 110: 9, 111: 7, 112: 21, 113: 14, 114: 6, 115: 4, 116: 8, 117: 11, 118: 14, 119: 1, 120: 7, 121: 4, 122: 35, 123: 11, 124: 2, 125: 5, 126: 3, 127: 3, 128: 0, 129: 1, 1